In [3]:
import numpy as np
import scipy.stats as st
import matplotlib.pyplot as plt
import seaborn as sns

In [18]:
def sde_euler(drift, diffusion, T, dt, x0):
    """
    Simulates the Euler-Maruyama method for a stochastic differential equation.

    Args:
        drift: The drift function of the SDE.
        diffusion: The diffusion function of the SDE.
        T: The final time.
        dt: The time step.
        x0: The initial value.

    Returns:
        A NumPy array of the simulated values.
    """

    x = np.zeros((int(T / dt) + 1, x0.size))
    x[0] = x0
    for i in range(1, len(x)):
        dW = st.norm(0, np.sqrt(dt)).rvs(size=x0.size)
        x[i] = x[i - 1] + drift(x[i - 1]) * dt + diffusion(x[i - 1]) * dW

    return x

In [5]:
drift = lambda x: np.mean(x) - x
diffusion = lambda x: 1
sde = sde_euler(drift, diffusion, 1, 1e-3, np.zeros(10))

In [79]:
def plot_trajectory(x, opt=True):
    """
    Plot the trajectories of X and X_bar.

    Args:
        obj: The object containing the data.
        opt: Whether to plot individual trajectories.
    """

    t = np.arange(0, 1+1e-3, 1e-3)
    xbar = np.mean(x, 1)
    fig, ax = plt.subplots(figsize=(14, 7))

    if opt:
        ax.plot(t, x, 'k')

    # Average Line
    ax.plot(t, xbar, 'green', linewidth=2)

    ax.set_xlabel('t')
    ax.set_ylabel(r'$X_{t}$')

    plt.xlim([0, 1])
    plt.show()

In [29]:
class Model:

    def __init__(self, N, alpha, sig):
        """Initialise variables"""
        self.dt = 1e-3
        self.t = np.arange(0, 1+self.dt, self.dt)
        self.N = N
        self.x0 = np.zeros(self.N)
        self.x = np.zeros((len(self.t), self.N))
        self.xbar = np.zeros(len(self.t))
        self.alpha = alpha
        self.sig = sig
        self.f = lambda x: alpha * (np.mean(x) - x)
        self.g = lambda x: sig
        self.eta = -0.7

    def simulate(self):
        """Simulate the model"""
        self.x = sde_euler(self.f, self.g, 1, self.dt, self.x0)

    def erdosrenyi(self, p):
        """
        Function to generate an Erdős–Rényi random graph.

        Args:
            obj: The object containing the graph.
            p: The probability of an edge being present.

        Returns:
            The object with the updated graph.
        """

        # Generate a random matrix with entries 0 or 1, where 1 indicates an edge.
        self.A = np.random.rand(self.N, self.N) < p

        # Make the matrix upper triangular.
        self.A = np.triu(self.A, k=1)

        # Add the identity matrix to the matrix.
        self.A = self.A + self.A.T + np.eye(self.N)

    def network(self):
        self.f = lambda x: self.alpha / self.N * (np.matmul(self.A, x) - x*np.sum(self.A, axis=1))

    def plot_trajectory(self, opt=True):
        self.xbar = np.mean(self.x, 1)
        fig, ax = plt.subplots(figsize=(14, 7))

        if opt:
            ax.plot(self.t, self.x, 'k')

        # Average Line
        ax.plot(self.t, self.xbar, 'green', linewidth=2)

        ax.set_xlabel('t')
        ax.set_ylabel(r'$X_{t}$')

        plt.xlim([0, 1])
        plt.show()

In [30]:
obj = Model(10, 1, 1)
obj.erdosrenyi(0.5)
obj.network()
obj.simulate()